In [24]:
#####imports######

import requests
import sys 
import arcpy
import os
from tkinter import filedialog,simpledialog,Tk, Label, Entry, Button, StringVar, messagebox,  Toplevel, Listbox, MULTIPLE, Button, END, Scrollbar, RIGHT, Y, LEFT, BOTH

arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

print("Environment Ready")

Environment Ready


In [25]:
def CitySelector():
    root = Tk()
    root.withdraw()
    
    city_fc = filedialog.askopenfilename(title="Select City Boundary Feature")
    city_name = os.path.splitext(os.path.basename(city_fc))[0]
    
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    m = aprx.activeMap
    m.addDataFromPath(city_fc)

    print(f"City boundary loaded: {city_name}")
    return city_name, city_fc


#DEFINE CITY SELECTOR INPUT FUNCTION
def CitySelector():
    root = Tk()
    root.withdraw()

#ask for boundaries
    city_boundary_path = filedialog.askopenfilename(
        title="Select City Boundaries Shapefile",
        filetypes=[("Shapefiles","*.shp")]
    )
    if not city_boundary_path: 
        print("no file selected")
        return None
    print(f"Loaded boundary file: {city_boundary_path}")

#Ask for Cities Field
    cityfields = arcpy.ListFields(city_boundary_path)
    field_names = [f.name for f in cityfields]
    print("\nAvailable Fields:")
    for f in field_names:
        print(" -",f)

    city_field = input()
    "City Name Field",
    f"Enter the field that contains City Names"
    
    print(f"Using field {city_field}")
    root.destroy()
#Sort Cities
    
    city_names = set()
    with arcpy.da.SearchCursor(city_boundary_path, [city_field]) as cursor:
        for row in cursor:
            if row[0] not in (None, "", " "):
                city_names.add(row[0])
    city_names = sorted(city_names)
    
    print("\nCities Found:")
    for c in city_names:
        print(" -", c)

#Select City
    print("Select City")
    selected_city = input()
    f"Type the city name exactly as shown:\n\n{city_names}"
    print(f"{selected_city}")
    
    if selected_city not in city_names:
        print("Invalid city name. Cancelling.")
        return None

    print(f"Selected city: {selected_city}")

    # --- 5. Output location ---
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    m = aprx.activeMap
    project_folder = os.path.dirname(aprx.filePath)
    out_gdb = os.path.join(project_folder, "CitySelections.gdb")

    if not arcpy.Exists(out_gdb):
        arcpy.management.CreateFileGDB(project_folder, "CitySelections.gdb")

    out_fc = os.path.join(out_gdb, f"{selected_city}")

    # Overwrite if exists
    if arcpy.Exists(out_fc):
        arcpy.management.Delete(out_fc)

    # --- 6. Select chosen city ---
    where_clause = f"{city_field} = '{selected_city}'"
    arcpy.analysis.Select(
        in_features=city_boundary_path,
        out_feature_class=out_fc,
        where_clause=where_clause
    )

    # --- 7. Add selected city to map ---
    m.addDataFromPath(out_fc)
    print(f"City {selected_city} added to map at: {out_fc}")
    city_name = selected_city
    city_fc = out_fc
    return selected_city, city_fc
    

In [26]:
#DEFINE PARCEL SELECTOR INPUT FUNCTION
def ParcelSelector():
    root = Tk()
    root.withdraw()

    # ==========================================================
    # 1. Ask user for the parcel file (allows reselection freely)
    # ==========================================================
    parcel_path = filedialog.askopenfilename(
        title="Select Parcel Shapefile or Feature Class",
        filetypes=[("Shapefiles", "*.shp"), ("All Files", "*.*")]
    )

    if not parcel_path:
        print("No parcel file selected.")
        root.destroy()
        return None, None

    print(f"Loaded parcel file: {parcel_path}")

    # ==========================================================
    # 2. Read field list
    # ==========================================================
    fields = arcpy.ListFields(parcel_path)
    field_names = [f.name for f in fields]

    # ==========================================================
    # 3. Create popup window for field selection
    # ==========================================================
    field_window = Toplevel(root)
    field_window.title("Select Fields")
    field_window.geometry("500x500")

    # Multi-select listbox
    listbox = Listbox(field_window, selectmode=MULTIPLE, width=40)
    listbox.pack(side=LEFT, fill=BOTH, expand=True)

    # Scrollbar
    scrollbar = Scrollbar(field_window)
    scrollbar.pack(side=RIGHT, fill=Y)

    listbox.config(yscrollcommand=scrollbar.set)
    scrollbar.config(command=listbox.yview)

    for name in field_names:
        listbox.insert(END, name)

    selected_fields = []

    # ==========================================================
    # 4. Confirm button (safe destroy + valid return)
    # ==========================================================
    def confirm_selection():
        idxs = listbox.curselection()
        if not idxs:
            print("No fields selected! Please select at least one.")
            return

        selected_fields.extend(field_names[i] for i in idxs)

        # destroy only the Toplevel, not the Tk root yet
        field_window.destroy()

    Button(field_window, text="Confirm Selection", command=confirm_selection).pack(pady=10)

    # Wait for the field window to close
    root.wait_window(field_window)

    # ==========================================================
    # 5. Clean up & return
    # ==========================================================
    root.destroy()

    if not selected_fields:
        print("No fields selected. Cancelled.")
        return None, None

    print(f"Fields selected: {selected_fields}")

    return parcel_path, selected_fields
    selected_field = selected_fields


In [27]:
#DEFINE CENTERLINE SELECTOR INPUT FUNCTION
def CenterlineSelector():
    root = Tk()
    root.withdraw()

#CENTERLINE PATH EASY

    center_path = filedialog.askopenfilename(
        title="Select Centerline Shapefile or Feature Class",
        filetypes=[("Shapefiles", "*.shp"), ("All Files", "*.*")]
    )

    if not center_path:
        print("No centerline file selected.")
        root.destroy()
        return None

    print(f"Loaded centerline file: {center_path}")
    return center_path


In [28]:
##RealSelectorHours

#DEFINE CITY SELECTOR INPUT FUNCTION
def CitySelector():
    root = Tk()
    root.withdraw()

#ask for boundaries
    city_boundary_path = filedialog.askopenfilename(
        title="Select City Boundaries Shapefile",
        filetypes=[("Shapefiles","*.shp")]
    )
    if not city_boundary_path: 
        print("no file selected")
        return None
    print(f"Loaded boundary file: {city_boundary_path}")

#Ask for Cities Field
    cityfields = arcpy.ListFields(city_boundary_path)
    field_names = [f.name for f in cityfields]
    print("\nAvailable Fields:")
    for f in field_names:
        print(" -",f)

    city_field = input()
    "City Name Field",
    f"Enter the field that contains City Names"
    
    print(f"Using field {city_field}")
    root.destroy()
#Sort Cities
    
    city_names = set()
    with arcpy.da.SearchCursor(city_boundary_path, [city_field]) as cursor:
        for row in cursor:
            if row[0] not in (None, "", " "):
                city_names.add(row[0])
    city_names = sorted(city_names)
    
    print("\nCities Found:")
    for c in city_names:
        print(" -", c)

#Select City
    print("Select City")
    selected_city = input()
    f"Type the city name exactly as shown:\n\n{city_names}"
    print(f"{selected_city}")
    
    if selected_city not in city_names:
        print("Invalid city name. Cancelling.")
        return None

    print(f"Selected city: {selected_city}")

    # --- 5. Output location ---
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    m = aprx.activeMap
    project_folder = os.path.dirname(aprx.filePath)
    out_gdb = os.path.join(project_folder, "CitySelections.gdb")

    if not arcpy.Exists(out_gdb):
        arcpy.management.CreateFileGDB(project_folder, "CitySelections.gdb")

    out_fc = os.path.join(out_gdb, f"{selected_city}")

    # Overwrite if exists
    if arcpy.Exists(out_fc):
        arcpy.management.Delete(out_fc)

    # --- 6. Select chosen city ---
    where_clause = f"{city_field} = '{selected_city}'"
    arcpy.analysis.Select(
        in_features=city_boundary_path,
        out_feature_class=out_fc,
        where_clause=where_clause
    )

    city_name = selected_city
    city_fc = out_fc
   
    return selected_city, city_fc
    #DEFINE PARCEL SELECTOR INPUT FUNCTION
def ParcelSelector():
    root = Tk()
    root.withdraw()

    # ==========================================================
    # 1. Ask user for the parcel file (allows reselection freely)
    # ==========================================================
    parcel_path = filedialog.askopenfilename(
        title="Select Parcel Shapefile or Feature Class",
        filetypes=[("Shapefiles", "*.shp"), ("All Files", "*.*")]
    )

    if not parcel_path:
        print("No parcel file selected.")
        root.destroy()
        return None, None

    print(f"Loaded parcel file: {parcel_path}")

    # ==========================================================
    # 2. Read field list
    # ==========================================================
    fields = arcpy.ListFields(parcel_path)
    field_names = [f.name for f in fields]

    # ==========================================================
    # 3. Create popup window for field selection
    # ==========================================================
    field_window = Toplevel(root)
    field_window.title("Select Fields")
    field_window.geometry("500x500")

    # Multi-select listbox
    listbox = Listbox(field_window, selectmode=MULTIPLE, width=40)
    listbox.pack(side=LEFT, fill=BOTH, expand=True)

    # Scrollbar
    scrollbar = Scrollbar(field_window)
    scrollbar.pack(side=RIGHT, fill=Y)

    listbox.config(yscrollcommand=scrollbar.set)
    scrollbar.config(command=listbox.yview)

    for name in field_names:
        listbox.insert(END, name)

    selected_fields = []

    # ==========================================================
    # 4. Confirm button (safe destroy + valid return)
    # ==========================================================
    def confirm_selection():
        idxs = listbox.curselection()
        if not idxs:
            print("No fields selected! Please select at least one.")
            return

        selected_fields.extend(field_names[i] for i in idxs)

        # destroy only the Toplevel, not the Tk root yet
        field_window.destroy()

    Button(field_window, text="Confirm Selection", command=confirm_selection).pack(pady=10)

    # Wait for the field window to close
    root.wait_window(field_window)

    # ==========================================================
    # 5. Clean up & return
    # ==========================================================
    root.destroy()

    if not selected_fields:
        print("No fields selected. Cancelled.")
        return None, None

    print(f"Fields selected: {selected_fields}")

    return parcel_path, selected_fields
    selected_field = selected_fields

    
#DEFINE CENTERLINE SELECTOR INPUT FUNCTION
def CenterlineSelector():
    root = Tk()
    root.withdraw()

#CENTERLINE PATH EASY

    center_path = filedialog.askopenfilename(
        title="Select Centerline Shapefile or Feature Class",
        filetypes=[("Shapefiles", "*.shp"), ("All Files", "*.*")]
    )

    if not center_path:
        print("No centerline file selected.")
        root.destroy()
        return None

    print(f"Loaded centerline file: {center_path}")
    return center_path



In [29]:
def TopographySlopeFromContours(city_name, city_fc):
    import arcpy
    from tkinter import Tk, filedialog
    import os

    arcpy.CheckOutExtension("Spatial")
    arcpy.CheckOutExtension("3D")

    root = Tk()
    root.withdraw()

    contour_fc = filedialog.askopenfilename(
        title="Select contour shapefile",
        filetypes=[("Shapefile", "*.shp")]
    )

    print("Contour file selected:")
    print(contour_fc)

    fields = [f.name for f in arcpy.ListFields(contour_fc)]
    print("Available fields:")
    for f in fields:
        print(f)

    elev_field = input("Type the elevation field name exactly as shown above: ")

    project_folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder

    clipped_contours = os.path.join(project_folder, f"{city_name}_ContoursTemp.shp")
    arcpy.analysis.Clip(contour_fc, city_fc, clipped_contours)

    city_dem = os.path.join(project_folder, f"{city_name}_DEM_Temp.tif")

    arcpy.ddd.TopoToRaster(
        [[clipped_contours, elev_field]],
        city_dem,
        10
    )

    city_slope = os.path.join(project_folder, f"{city_name}_Slope_Temp.tif")
    arcpy.sa.Slope(city_dem, "DEGREE").save(city_slope)

    master_dem = os.path.join(project_folder, "Master_DEM.tif")
    master_slope = os.path.join(project_folder, "Master_Slope.tif")

    if arcpy.Exists(master_dem):
        arcpy.management.Mosaic(
            city_dem,
            master_dem,
            mosaic_type="LAST"
        )
    else:
        arcpy.management.CopyRaster(city_dem, master_dem)

    if arcpy.Exists(master_slope):
        arcpy.management.Mosaic(
            city_slope,
            master_slope,
            mosaic_type="LAST"
        )
    else:
        arcpy.management.CopyRaster(city_slope, master_slope)

    aprx = arcpy.mp.ArcGISProject("CURRENT")
    m = aprx.activeMap

    m.addDataFromPath(master_dem)
    m.addDataFromPath(master_slope)

    print("Master DEM and Master Slope updated with city:")
    print(city_name)

    return master_slope


In [19]:
def TopographySlopeFromContours(city_name, city_fc):
    import arcpy
    from tkinter import Tk, filedialog
    import os

    arcpy.CheckOutExtension("Spatial")
    arcpy.CheckOutExtension("3D")

    # Select contour shapefile
    root = Tk()
    root.withdraw()

    contour_fc = filedialog.askopenfilename(
        title="Select contour shapefile",
        filetypes=[("Shapefile", "*.shp")]
    )

    print("Contour file selected:")
    print(contour_fc)

    # Ask for elevation field
    fields = [f.name for f in arcpy.ListFields(contour_fc)]
    print("Available fields:")
    for f in fields:
        print(f)

    elev_field = input("Type the elevation field name exactly as shown above: ")

    project_folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder

    # Temporary city contours
    clipped_contours = os.path.join(project_folder, f"{city_name}_ContoursTemp.shp")
    arcpy.analysis.Clip(contour_fc, city_fc, clipped_contours)

    # Temporary city DEM
    city_dem = os.path.join(project_folder, f"{city_name}_DEM_Temp.tif")
    arcpy.ddd.TopoToRaster([[clipped_contours, elev_field]], city_dem, 10)

    # Temporary city slope (RAW)
    city_slope = os.path.join(project_folder, f"{city_name}_Slope_Temp.tif")
    arcpy.sa.Slope(city_dem, "DEGREE").save(city_slope)

    # ✅ ✅ ✅ NEW: Categorized slope (5 classes)
    city_slope_cat = os.path.join(project_folder, f"{city_name}_Slope_CAT.tif")

    remap = arcpy.sa.RemapRange([
        [0, 2, 1],     # Flat
        [2, 5, 2],     # Gentle
        [5, 15, 3],    # Moderate
        [15, 30, 4],   # Steep
        [30, 90, 5]    # Very Steep
    ])

    arcpy.sa.Reclassify(city_slope, "Value", remap).save(city_slope_cat)

    # Master rasters
    master_dem = os.path.join(project_folder, "Master_DEM.tif")
    master_slope = os.path.join(project_folder, "Master_Slope.tif")

    # Update master DEM (unchanged)
    if arcpy.Exists(master_dem):
        arcpy.management.Mosaic(city_dem, master_dem, mosaic_type="LAST")
    else:
        arcpy.management.CopyRaster(city_dem, master_dem)

    # ✅ ✅ ✅ Update master SLOPE USING CATEGORICAL RASTER
    if arcpy.Exists(master_slope):
        arcpy.management.Mosaic(city_slope_cat, master_slope, mosaic_type="LAST")
    else:
        arcpy.management.CopyRaster(city_slope_cat, master_slope)

    # Add master rasters to map
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    m = aprx.activeMap

    m.addDataFromPath(master_dem)
    m.addDataFromPath(master_slope)

    print("Master DEM and categorical slope updated")

    # Clean up temporary city topo files
    temp_files = [
        clipped_contours,
        city_dem,
        city_slope,
        city_slope_cat
    ]

    for f in temp_files:
        if arcpy.Exists(f):
            arcpy.management.Delete(f)

    print("Temporary topography files cleaned up")

    return master_slope


In [37]:
def RunLandSelector():
    import arcpy
    import os
    from tkinter import filedialog
    import time

    city_name, city_fc = CitySelector()
    parcel_fc, parcel_field = ParcelSelector()
    road_fc = CenterlineSelector()

    output_folder = filedialog.askdirectory(
        title="Select folder to save final land output"
    )

    output_name = arcpy.ValidateTableName(
        f"{city_name}_FinalLand.shp",
        output_folder
    )

    final_output = os.path.join(output_folder, output_name)

    print("Updating master DEM and slope from contours...")
    slope_raster = TopographySlopeFromContours(city_name, city_fc)

    # Temporary working files (disk-based to avoid crashes)
    clipped_parcels = os.path.join(output_folder, f"{city_name}_parcels_clipped.shp")
    clipped_roads = os.path.join(output_folder, f"{city_name}_roads_clipped.shp")
    road_buffer = os.path.join(output_folder, f"{city_name}_road_buffer.shp")
    erased_land = os.path.join(output_folder, f"{city_name}_land_gaps_all.shp")

    # Clip parcels and roads
    arcpy.analysis.Clip(parcel_fc, city_fc, clipped_parcels)
    arcpy.analysis.Clip(road_fc, city_fc, clipped_roads)

    # Repair geometry safely (now the files always exist)
    arcpy.management.RepairGeometry(clipped_parcels)
    arcpy.management.RepairGeometry(clipped_roads)

    # Buffer roads
    fields = [f.name for f in arcpy.ListFields(clipped_roads)]

    if "WIDTH" in fields:
        arcpy.analysis.Buffer(clipped_roads, road_buffer, "!WIDTH! + 10")
    else:
        arcpy.analysis.Buffer(clipped_roads, road_buffer, "25 Feet")

    # Erase roads from parcels (keeps all parcel sizes)
    arcpy.analysis.Erase(clipped_parcels, road_buffer, erased_land)

    # Save only the final land output for the city
    arcpy.management.CopyFeatures(erased_land, final_output)

    # Add final output to map

    import time
# Make sure the file actually exists before adding it
    if arcpy.Exists(final_output):
        time.sleep(1)  # short pause so ArcGIS can finish writing the file

        aprx = arcpy.mp.ArcGISProject("CURRENT")
        m = aprx.activeMap

        try:
            m.addDataFromPath(final_output)
            print("Final land layer added to the map.")
        except:
            print("Final land file was created, but ArcGIS could not add it automatically.")
            print("You can add it manually from this location:")
            print(final_output)
    else:
        print("Final output file was not found on disk:")
        print(final_output)


        print("Final microdevelopment land saved for city:")
        print(city_name)
        print(final_output)

    return final_output


In [51]:
def RunLandSelector():
    import arcpy
    import os
    import time
    from tkinter import filedialog

    city_name, city_fc = CitySelector()
    parcel_fc, parcel_field = ParcelSelector()
    road_fc = CenterlineSelector()

    output_folder = filedialog.askdirectory(
        title="Select folder to save final land output"
    )

    output_name = arcpy.ValidateTableName(
        f"{city_name}_FinalLand.shp",
        output_folder
    )

    final_output = os.path.join(output_folder, output_name)

    print("Updating master DEM and slope from contours...")
    slope_raster = TopographySlopeFromContours(city_name, city_fc)

    # Temporary working files (disk-based)
    clipped_parcels = os.path.join(output_folder, f"{city_name}_parcels_temp.shp")
    clipped_roads = os.path.join(output_folder, f"{city_name}_roads_temp.shp")
    road_buffer = os.path.join(output_folder, f"{city_name}_road_buffer_temp.shp")
    erased_land = os.path.join(output_folder, f"{city_name}_land_gaps_temp.shp")

    # Clip parcels and roads
    arcpy.analysis.Clip(parcel_fc, city_fc, clipped_parcels)
    arcpy.analysis.Clip(road_fc, city_fc, clipped_roads)

    # Repair geometry
    arcpy.management.RepairGeometry(clipped_parcels)
    arcpy.management.RepairGeometry(clipped_roads)

    # Buffer roads
    fields = [f.name for f in arcpy.ListFields(clipped_roads)]

    if "WIDTH" in fields:
        arcpy.analysis.Buffer(clipped_roads, road_buffer, "!WIDTH! + 10")
    else:
        arcpy.analysis.Buffer(clipped_roads, road_buffer, "25 Feet")

    # Erase roads from parcels
    arcpy.analysis.Erase(clipped_parcels, road_buffer, erased_land)


    #
    arcpy.analysis.PairwiseErase(
    in_features=final_output,
    erase_features=city_fc,
    out_feature_class= f"{city_name}_UnownedLand",
    cluster_tolerance=None
    )

    # Save final output
    arcpy.management.CopyFeatures(erased_land, final_output)

    # Safe add to map
    time.sleep(1)

    if arcpy.Exists(final_output):
        aprx = arcpy.mp.ArcGISProject("CURRENT")
        m = aprx.activeMap

        try:
            m.addDataFromPath(final_output)
            print("Final land added to map")
        except:
            print("Final land saved but could not be added automatically")
            print(final_output)
    else:
        print("Final output not found on disk:")
        print(final_output)

    # Automatic cleanup of temp vector files
    temp_files = [
        clipped_parcels,
        clipped_roads,
        road_buffer,
        erased_land
    ]

    for f in temp_files:
        if arcpy.Exists(f):
            arcpy.management.Delete(f)

    print("Temporary parcel and road files cleaned up")

    print("Final microdevelopment land saved for city:")
    print(city_name)
    print(final_output)

    return final_output


In [52]:
RunLandSelector()

Loaded boundary file: C:/Users/sible041/Desktop/Data/Census2020CTUs.shp

Available Fields:
 - FID
 - Shape
 - CTU_ID
 - CTU_NAME
 - CTU_ID_CEN
 - CTU_CODE
 - CTU_TYPE
 - ABC_SORT
 - FIVE_COLOR
 - Shape_Leng
 - Shape_Area


 CTU_NAME


Using field CTU_NAME

Cities Found:
 - Afton
 - Andover
 - Anoka
 - Apple Valley
 - Arden Hills
 - Bayport
 - Baytown Twp.
 - Belle Plaine
 - Belle Plaine Twp.
 - Benton Twp.
 - Bethel
 - Birchwood Village
 - Blaine
 - Blakeley Twp.
 - Bloomington
 - Brooklyn Center
 - Brooklyn Park
 - Burnsville
 - Camden Twp.
 - Carver
 - Castle Rock Twp.
 - Cedar Lake Twp.
 - Centerville
 - Champlin
 - Chanhassen
 - Chaska
 - Circle Pines
 - Coates
 - Cologne
 - Columbia Heights
 - Columbus
 - Coon Rapids
 - Corcoran
 - Cottage Grove
 - Credit River Twp.
 - Crystal
 - Dahlgren Twp.
 - Dayton
 - Deephaven
 - Dellwood
 - Denmark Twp.
 - Douglas Twp.
 - Eagan
 - East Bethel
 - Eden Prairie
 - Edina
 - Elko New Market
 - Empire Twp.
 - Eureka Twp.
 - Excelsior
 - Falcon Heights
 - Farmington
 - Forest Lake
 - Fort Snelling (unorg.)
 - Fridley
 - Gem Lake
 - Golden Valley
 - Grant
 - Greenfield
 - Greenvale Twp.
 - Greenwood
 - Grey Cloud Island Twp.
 - Ham Lake
 - Hamburg
 - Hampton
 - Hampton Twp.
 - H

 Hopkins


Hopkins
Selected city: Hopkins
Loaded parcel file: C:/Users/sible041/Desktop/Data/County_Parcels.shp
Fields selected: ['FID', 'Shape', 'OBJECTID', 'PID', 'PID_TEXT', 'DIV_STATUS', 'FEATURECOD', 'STATE_CD', 'TORRENS_TY', 'HOUSE_NO', 'FRAC_HOUSE', 'STREET_NM', 'CONDO_NO', 'MAILING_MU', 'MAILING__1', 'ZIP_CD', 'MULTI_ADDR', 'OWNER_NM', 'TAXPAYER_N', 'TAXPAYER_1', 'TAXPAYER_2', 'TAXPAYER_3', 'MUNIC_CD', 'MUNIC_NM', 'SCHOOL_DIS', 'WATERSHED_', 'SEWER_DIST', 'TIF_PROJEC', 'PROPERTY_S', 'FORFEIT_LA', 'CO_OP_IND', 'PRI_SEC_CO', 'ABBREV_ADD', 'ADDITION_N', 'LOT', 'BLOCK', 'METES_BNDS', 'METES_BN_1', 'METES_BN_2', 'METES_BN_3', 'MORE_METES', 'ABSTR_TORR', 'BUILD_YR', 'SALE_DATE', 'SALE_PRICE', 'SALE_CODE', 'SALE_CODE_', 'PARCEL_ARE', 'MKT_VAL_TO', 'TAXABLE_VA', 'NET_IMPRV_', 'TOT_NET_TA', 'TOT_SPEC_T', 'TAX_TOT', 'NET_TAX_PD', 'TOT_PENALT', 'EARLIEST_D', 'COMP_JUDG_', 'MTG_CO_NBR', 'GR_ACRE_OP', 'PETITION_R', 'TAX_ADJ_PE', 'DIV_PEND_I', 'PR_TYP_CD1', 'PR_TYP_NM1', 'HMSTD_CD1', 'OWNER_PCT1', 'CON

Type the elevation field name exactly as shown above:  Elevation


Master DEM and categorical slope updated
Temporary topography files cleaned up


<class 'NameError'>: name 'UnownedLand' is not defined

In [ ]:
#SPACE ANALYZER

In [45]:
# Step 1, Look at the price of the dfd
def FindUndevelopedParcels(parcel_fc, output_folder, city_name):
    import arcpy
    import os

    print("Available parcel fields:")
    fields = [f.name for f in arcpy.ListFields(parcel_fc)]
    for f in fields:
        print(f)

    sale_field = input("Type the field name for SALE VALUE (0 = never sold): ")
    tax_field = input("Type the field name for TOTAL TAX (0 = no tax): ")
    value_field = input("Type the field name for TOTAL PROPERTY VALUE (0 = no value): ")

    output_name = arcpy.ValidateTableName(
        f"{city_name}_UndevelopedOrEasement.shp",
        output_folder
    )

    output_path = os.path.join(output_folder, output_name)

    where_clause = f"""
    ({sale_field} = 0 OR {sale_field} IS NULL)
    OR
    ({tax_field} = 0 OR {tax_field} IS NULL)
    OR
    ({value_field} = 0 OR {value_field} IS NULL)
    """

    print("Selecting parcels that meet undeveloped/easement criteria...")

    arcpy.management.MakeFeatureLayer(parcel_fc, "parcel_layer")
    arcpy.management.SelectLayerByAttribute(
        "parcel_layer",
        "NEW_SELECTION",
        where_clause
    )

    arcpy.management.CopyFeatures("parcel_layer", output_path)

    print("Undeveloped / easement parcel layer created:")
    print(output_path)

    aprx = arcpy.mp.ArcGISProject("CURRENT")
    m = aprx.activeMap
    m.addDataFromPath(output_path)

    return output_path


In [46]:
undeveloped_layer = FindUndevelopedParcels(
    parcel_fc,
    output_folder,
    city_name
)


<class 'NameError'>: name 'parcel_fc' is not defined